Goal: customer segment based on department and asile 

Method: LDA. The reason for choosing LDA is we can treat department name and aisle name as words and we can grasp topic based on frequency of these words. I want to segment customers into 5 groups. 

In [1]:
import pandas as pd
import gensim
from gensim import corpora, models, similarities
from gensim.models import LdaMulticore

import numpy as np

import pickle

## Cluster Segment

### Based on Department

#### Build Model

In [2]:
def print_top_product(top_words_list, cluster_num, group_df, groupby_name):
    prior_department_group = prior_data_da[prior_data_da['user_id'].isin(group_df[cluster_num].user_id.values)]
    res = prior_department_group.groupby([groupby_name, 'product_name'])['user_id'].count().reset_index()
    
    for i, item in enumerate(top_words_list[cluster_num]):
        print 'the most popular product in ' + str(item) + ':'
        top_product = pd.DataFrame(res[res[groupby_name] == top_words_list[cluster_num][i]].groupby('product_name', as_index = False)['user_id'].sum()).sort_values('user_id', ascending = False)[0:5].product_name.values
        print top_product
        print 

In [3]:
prior_data_da = pd.read_csv('/Users/mengruiyin/Desktop/project/prior_data_da.csv')

In [4]:
prior_data_da.head(10)

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,beverages,soft drinks
1,1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,dairy eggs,soy lactosefree
2,2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,snacks,popcorn jerky
3,3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,snacks,popcorn jerky
4,4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,household,paper goods
5,5,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,beverages,soft drinks
6,6,2398795,1,prior,2,3,7,15.0,10258,2,0,Pistachios,117,19,snacks,nuts seeds dried fruit
7,7,2398795,1,prior,2,3,7,15.0,12427,3,1,Original Beef Jerky,23,19,snacks,popcorn jerky
8,8,2398795,1,prior,2,3,7,15.0,13176,4,0,Bag of Organic Bananas,24,4,produce,fresh fruits
9,9,2398795,1,prior,2,3,7,15.0,26088,5,1,Aged White Cheddar Popcorn,23,19,snacks,popcorn jerky


In [5]:
department_list = prior_data_da.groupby('user_id').agg({'department': lambda x: list(x)})
#department_list = pd.DataFrame(department_list.reset_index())
#department_list.head(10)

In [6]:
corps = department_list.department.tolist()

In [7]:
len(corps)

206209

In [8]:
dictionary = corpora.dictionary.Dictionary(corps)

In [9]:
corpus = [dictionary.doc2bow(corps) for corps in corps]

In [ ]:
# lda = models.LdaModel(corpus, num_topics=10, 
#                             id2word=dictionary, 
#                             update_every=5, 
#                             chunksize=70000, 
#                             passes=100)

In [9]:
# num_topics = 5
# chunksize = 70000
# passes = 20
# iterations = 100
# eval_every = None  # Don't evaluate model perplexity, takes too much time.

# # Make a index to word dictionary.
# temp = dictionary[0]  # This is only to "load" the dictionary.
# id2word = dictionary.id2token
# #alpha='auto', eta='auto', \
# %time model = LdaMulticore(corpus=corpus, id2word=id2word, chunksize=chunksize, \
#                        iterations=iterations, num_topics=num_topics, \
#                        passes=passes, eval_every=eval_every, eta='auto', workers = 4)

CPU times: user 18min 21s, sys: 29min 32s, total: 47min 54s
Wall time: 48min 8s


In [81]:
# save the model to disk
filename = 'LDA_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [10]:
#load the model from disk
filename = 'LDA_model.sav'
model = pickle.load(open(filename, 'rb'))

#### Print Top 10 Departments for Each Group

In [11]:
model.show_topics()

[(0,
  u'0.555*"produce" + 0.124*"dairy eggs" + 0.052*"pantry" + 0.044*"snacks" + 0.041*"frozen" + 0.034*"beverages" + 0.028*"deli" + 0.028*"canned goods" + 0.023*"meat seafood" + 0.021*"bakery"'),
 (1,
  u'0.412*"beverages" + 0.116*"household" + 0.104*"dairy eggs" + 0.068*"snacks" + 0.061*"personal care" + 0.059*"frozen" + 0.055*"pantry" + 0.029*"produce" + 0.028*"breakfast" + 0.020*"bakery"'),
 (2,
  u'0.491*"snacks" + 0.078*"frozen" + 0.071*"produce" + 0.065*"beverages" + 0.045*"breakfast" + 0.045*"alcohol" + 0.041*"pantry" + 0.038*"dairy eggs" + 0.025*"pets" + 0.024*"deli"'),
 (3,
  u'0.217*"produce" + 0.158*"frozen" + 0.103*"canned goods" + 0.099*"pantry" + 0.089*"babies" + 0.070*"dairy eggs" + 0.065*"deli" + 0.049*"dry goods pasta" + 0.046*"snacks" + 0.023*"international"'),
 (4,
  u'0.478*"dairy eggs" + 0.109*"bakery" + 0.068*"frozen" + 0.056*"dry goods pasta" + 0.051*"pantry" + 0.047*"produce" + 0.045*"meat seafood" + 0.038*"breakfast" + 0.034*"deli" + 0.027*"canned goods"')]

I created 5 groups for customers. Here printed 10 top department for each group of customers. Since there are not many departments, we can see the top 10 departments for each group are very similar.

In [12]:
topics_matrix = model.show_topics(formatted=False, num_words=10)
topics_matrix = np.array(topics_matrix, dtype=object)

In [13]:
len(topics_matrix)

5

In [14]:
for i in range(len(topics_matrix)):
    print [str(topics_matrix[i][1][index][0]) for index in range(10)]
    print

['produce', 'dairy eggs', 'pantry', 'snacks', 'frozen', 'beverages', 'deli', 'canned goods', 'meat seafood', 'bakery']

['beverages', 'household', 'dairy eggs', 'snacks', 'personal care', 'frozen', 'pantry', 'produce', 'breakfast', 'bakery']

['snacks', 'frozen', 'produce', 'beverages', 'breakfast', 'alcohol', 'pantry', 'dairy eggs', 'pets', 'deli']

['produce', 'frozen', 'canned goods', 'pantry', 'babies', 'dairy eggs', 'deli', 'dry goods pasta', 'snacks', 'international']

['dairy eggs', 'bakery', 'frozen', 'dry goods pasta', 'pantry', 'produce', 'meat seafood', 'breakfast', 'deli', 'canned goods']



In [15]:
docs_topic = []

for i in range(len(corpus)):
    a = model.get_document_topics(corpus[i])
    a.sort(key = lambda x: x[1],reverse=True)
    docs_topic.append(a[0][0])

In [16]:
department_list['topic'] = docs_topic
department_list = department_list.reset_index()

In [17]:
department_list.head(100)

,user_id,department,topic
0,1,"[beverages, dairy eggs, snacks, snacks, househ...",2
1,2,"[snacks, produce, deli, pantry, snacks, snacks...",2
2,3,"[produce, dairy eggs, produce, produce, dairy ...",0
3,4,"[bakery, beverages, frozen, household, alcohol...",1
4,5,"[pantry, international, produce, produce, prod...",0
5,6,"[produce, meat seafood, produce, produce, meat...",0
6,7,"[dairy eggs, produce, bakery, produce, dairy e...",0
7,8,"[produce, produce, produce, pantry, produce, p...",0
8,9,"[dairy eggs, pantry, produce, dairy eggs, dair...",4
9,10,"[produce, produce, produce, produce, produce, ...",0


In [18]:
department_list.to_csv('department_list.csv')

#### Printed Popular Products for Each Group based on Department

In [19]:
grouped = department_list.groupby('topic')
department_group = {}
#user_group = {}
for i in range(len(topics_matrix)):
    department_group[i] = grouped.get_group(i)
    #user_group[i] = department_group[i].user_id

In [20]:
top_words = [['produce', 'dairy eggs', 'pantry', 'snacks', 'frozen', 'beverages', 'deli', 'canned goods', 'meat seafood', 'bakery'],
['beverages', 'household', 'dairy eggs', 'snacks', 'personal care', 'frozen', 'pantry', 'produce', 'breakfast', 'bakery'],
['snacks', 'frozen', 'produce', 'beverages', 'breakfast', 'alcohol', 'pantry', 'dairy eggs', 'pets', 'deli'],
['produce', 'frozen', 'canned goods', 'pantry', 'babies', 'dairy eggs', 'deli', 'dry goods pasta', 'snacks', 'international'],
['dairy eggs', 'bakery', 'frozen', 'dry goods pasta', 'pantry', 'produce', 'meat seafood', 'breakfast', 'deli', 'canned goods']]

In [ ]:
['produce', 'dairy eggs', 'pantry', 'snacks', 'frozen', 'beverages', 'deli', 'canned goods', 'meat seafood', 'bakery']
()
['beverages', 'household', 'dairy eggs', 'snacks', 'personal care', 'frozen', 'pantry', 'produce', 'breakfast', 'bakery']
()
['snacks', 'frozen', 'produce', 'beverages', 'breakfast', 'alcohol', 'pantry', 'dairy eggs', 'pets', 'deli']
()
['produce', 'frozen', 'canned goods', 'pantry', 'babies', 'dairy eggs', 'deli', 'dry goods pasta', 'snacks', 'international']
()
['dairy eggs', 'bakery', 'frozen', 'dry goods pasta', 'pantry', 'produce', 'meat seafood', 'breakfast', 'deli', 'canned goods']
()

#### Popular Products for Group0

In [83]:
department_group[0]

,user_id,department,topic
2,3,"[produce, dairy eggs, produce, produce, dairy ...",0
4,5,"[pantry, international, produce, produce, prod...",0
5,6,"[produce, meat seafood, produce, produce, meat...",0
6,7,"[dairy eggs, produce, bakery, produce, dairy e...",0
7,8,"[produce, produce, produce, pantry, produce, p...",0
9,10,"[produce, produce, produce, produce, produce, ...",0
11,12,"[produce, produce, pantry, produce, pets, pets...",0
15,16,"[produce, produce, produce, produce, produce, ...",0
17,18,"[personal care, dairy eggs, beverages, persona...",0
21,22,"[produce, produce, produce, dairy eggs, dairy ...",0


In [119]:
print_top_product(top_words, 0, department_group)

the most popular product in produce:
['Banana' 'Bag of Organic Bananas' 'Organic Strawberries'
 'Organic Baby Spinach' 'Organic Hass Avocado']

the most popular product in dairy eggs:
['Organic Whole Milk' 'Organic Unsweetened Almond Milk'
 'Organic Half & Half' 'Half & Half' 'Organic Whole String Cheese']

the most popular product in pantry:
['Extra Virgin Olive Oil' 'Creamy Almond Butter'
 'Organic Creamy Peanut Butter' 'Italian Extra Virgin Olive Oil'
 'Organic Ketchup']

the most popular product in snacks:
['Lightly Salted Baked Snap Pea Crisps' 'Original Veggie Straws'
 'Sea Salt Pita Chips' 'Organic Whole Cashews' 'Whole Almonds']

the most popular product in frozen:
['Blueberries' 'Organic Whole Strawberries'
 'Frozen Organic Wild Blueberries' 'Organic Broccoli Florets'
 'Pineapple Chunks']

the most popular product in beverages:
['Sparkling Water Grapefruit' 'Spring Water' '100% Raw Coconut Water'
 'Lime Sparkling Water' 'Italian Sparkling Mineral Water']

the most popular prod

Based on top 10 key words printed for each department, customers from this group would like to buy products from produce, dairy eggs, pantry, snacks, frozen, beverages, deli, canned goods, meat seafood and bakery departments. The faviorite products of this group are products in produce department.

#### Popular Products for Group1

In [84]:
department_group[1]

,user_id,department,topic
3,4,"[bakery, beverages, frozen, household, alcohol...",1
10,11,"[dairy eggs, dairy eggs, dairy eggs, deli, pro...",1
16,17,"[beverages, beverages, alcohol, beverages, dai...",1
18,19,"[snacks, frozen, household, bakery, personal c...",1
23,24,"[beverages, pantry, beverages, bakery, househo...",1
25,26,"[dairy eggs, pantry, beverages, beverages, bev...",1
26,27,"[beverages, beverages, beverages, beverages, b...",1
29,30,"[beverages, snacks, snacks, beverages, beverag...",1
30,31,"[dairy eggs, dairy eggs, dairy eggs, beverages...",1
32,33,"[beverages, snacks, dairy eggs, beverages, pro...",1


In [122]:
print_top_product(top_words, 1, department_group)

the most popular product in beverages:
['Sparkling Water Grapefruit' 'Soda' 'Spring Water'
 'Sparkling Natural Mineral Water' 'Lime Sparkling Water']

the most popular product in household:
['100% Recycled Paper Towels' 'Ultra Soft Facial Tissues'
 'White Giant Paper Towel Rolls' 'Select-A-Size White Paper Towels'
 'Sustainably Soft Bath Tissue']

the most popular product in dairy eggs:
['Organic Half & Half' 'Half & Half' '2% Reduced Fat Milk'
 'Unsweetened Almondmilk' 'Whole Milk']

the most popular product in snacks:
['Mixed Fruit Fruit Snacks' 'Lightly Salted Baked Snap Pea Crisps'
 '100 Calorie  Per Bag Popcorn' 'Chocolate Chip Cookies'
 'Sea Salt & Vinegar Potato Chips']

the most popular product in personal care:
['Cotton Swabs' 'Lavender Hand Soap' 'Lemon Verbena Hand Soap'
 'Vitamin C Super Orange Dietary Supplement' 'Premium Epsom Salt']

the most popular product in frozen:
['Blueberries' 'Eggo Homestyle Waffles' 'Vanilla Ice Cream'
 'Macaroni & Cheese' 'Organic Broccoli Flor

Customers from this group would like to buy foods from beverages, household, dairy eggs, snacks, perosnal care, frozen, pantry, produce, breakfast and backery department. The favorite department of this group of coustomers is beverages department. Here I print the products from the top 10 department this group of customer would like to buy and we can find some specific products that this group of customer would like to buy but the first group of customer would not like to buy. For example, soda are popular for this group of customers. For products of dairy eggs, first group of customer would like to buy Organic Unsweetened Almond Milk while second group of customer would like to buy just Unsweetened Almondmilk. 2% Reduced Fat Milk are popular in second group but not in the first group. 

#### Popular Products for Group2

In [85]:
department_group[2]

,user_id,department,topic
0,1,"[beverages, dairy eggs, snacks, snacks, househ...",2
1,2,"[snacks, produce, deli, pantry, snacks, snacks...",2
14,15,"[snacks, snacks, snacks, beverages, snacks, sn...",2
19,20,"[snacks, snacks, produce, produce, produce, pr...",2
20,21,"[dairy eggs, dairy eggs, bakery, dairy eggs, s...",2
28,29,"[beverages, snacks, frozen, snacks, pantry, sn...",2
35,36,"[other, international, beverages, produce, int...",2
39,40,"[produce, bakery, produce, snacks, bakery, fro...",2
51,52,"[produce, snacks, snacks, beverages, beverages...",2
54,55,"[dairy eggs, snacks, snacks, frozen, beverages...",2


In [123]:
print_top_product(top_words, 2, department_group)

the most popular product in snacks:
['Trail Mix' 'Extra Fancy Unsalted Mixed Nuts'
 "Crunchy Oats 'n Honey Granola Bars" 'Mixed Fruit Fruit Snacks'
 'Chocolate Chip Cookies']

the most popular product in frozen:
['Blueberries' 'Macaroni & Cheese' 'Chocolate Ice Cream'
 'Eggo Homestyle Waffles' 'Ice Bag']

the most popular product in produce:
['Banana' 'Bag of Organic Bananas' 'Clementines' 'Organic Strawberries'
 'Strawberries']

the most popular product in beverages:
['Soda' 'Sparkling Water Grapefruit' 'Lime Sparkling Water'
 'Sparkling Mineral Water' 'Sparkling Natural Mineral Water']

the most popular product in breakfast:
['Honey Nut Cheerios' 'Cherrios Honey Nut' 'Cereal'
 'Nutri Grain Bars Multi Pack' 'with Crispy Almonds Cereal']

the most popular product in alcohol:
['Sauvignon Blanc' 'Beer' 'Chardonnay' 'India Pale Ale'
 'Cabernet Sauvignon']

the most popular product in pantry:
['Creamy Peanut Butter' 'Creamy Almond Butter' 'Extra Virgin Olive Oil'
 'Original Hot Sauce' 'Org

Customer from this group would like to buy products from snacks, frozen, produce, beverages, breakfast, alcohol, pantry, dairy egg, pets and deli departments.

#### Popular Products for Group3

In [86]:
department_group[3]

,user_id,department,topic
13,14,"[produce, bakery, pantry, canned goods, alcoho...",3
41,42,"[personal care, beverages, personal care, pers...",3
43,44,"[breakfast, produce, pantry, deli, deli, dairy...",3
46,47,"[pantry, dairy eggs, produce, dairy eggs, bake...",3
48,49,"[dry goods pasta, beverages, produce, beverage...",3
57,58,"[household, household, dry goods pasta, dry go...",3
58,59,"[dairy eggs, produce, produce, babies, produce...",3
73,74,"[produce, produce, international, produce, pro...",3
88,89,"[dairy eggs, dairy eggs, deli, produce, produc...",3
94,95,"[produce, produce, dairy eggs, produce, frozen...",3


In [130]:
print_top_product(top_words, 3, department_group, 'department')

the most popular product in produce:
['Banana' 'Bag of Organic Bananas' 'Organic Strawberries'
 'Organic Baby Spinach' 'Organic Hass Avocado']

the most popular product in frozen:
['Blueberries' 'Organic Broccoli Florets' 'Organic Whole Strawberries'
 'Macaroni & Cheese' 'Frozen Organic Wild Blueberries']

the most popular product in canned goods:
['Organic Black Beans' 'Organic Garbanzo Beans'
 'No Salt Added Black Beans' 'Black Beans' 'Organic AppleApple']

the most popular product in pantry:
['Extra Virgin Olive Oil' 'Creamy Peanut Butter' 'Organic Ketchup'
 'Organic Creamy Peanut Butter' 'Tomato Ketchup']

the most popular product in babies:
['Baby Food Stage 2 Blueberry Pear & Purple Carrot'
 'Spinach Peas & Pear Stage 2 Baby Food'
 'Broccoli & Apple Stage 2 Baby Food'
 'Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food'
 'Organic Pears, Peas and Broccoli Puree Stage 1']

the most popular product in dairy eggs:
['Organic Whole Milk' 'Organic Whole String

Customers from this group would like to buy foods from products from produce, frozen, canned goods, pantry, babies, dairy eggs, deli, dry goods pasta, sancks and interantioal departments. The favorite department of these customer is also produce department, which is the same as the first group of customer. However, the second favorite department of these customers is frozen department. 

#### Popular Products for Group4

In [87]:
department_group[4]

,user_id,department,topic
8,9,"[dairy eggs, pantry, produce, dairy eggs, dair...",4
12,13,"[pantry, produce, produce, dairy eggs, dairy e...",4
22,23,"[pantry, household, canned goods, household, d...",4
24,25,"[dry goods pasta, frozen, bakery, bakery, froz...",4
37,38,"[beverages, dairy eggs, frozen, dairy eggs, ho...",4
45,46,"[produce, snacks, dairy eggs, dairy eggs, froz...",4
50,51,"[meat seafood, dairy eggs, deli, produce, cann...",4
59,60,"[bakery, breakfast, breakfast, personal care, ...",4
62,63,"[pantry, dairy eggs, produce, frozen, frozen, ...",4
68,69,"[dairy eggs, frozen, dairy eggs, dairy eggs, d...",4


In [131]:
print_top_product(top_words, 4, department_group, 'department')

the most popular product in dairy eggs:
['Organic Whole Milk' 'Organic Half & Half' 'Half & Half'
 '2% Reduced Fat Milk' 'Whole Milk']

the most popular product in bakery:
['100% Whole Wheat Bread' 'Original Nooks & Crannies English Muffins'
 'Organic Bread with 21 Whole Grains' 'Plain Pre-Sliced Bagels'
 'Plain Bagels']

the most popular product in frozen:
['Blueberries' 'Organic Broccoli Florets'
 'Chicken & Maple Breakfast Sausage' 'Eggo Homestyle Waffles'
 'Organic Whole Strawberries']

the most popular product in dry goods pasta:
['Marinara Sauce' 'Spaghetti Pasta' 'Spaghetti'
 'Macaroni & Cheese Dinner Original Flavor' 'Penne Pasta']

the most popular product in pantry:
['Extra Virgin Olive Oil' 'Creamy Peanut Butter' 'Strawberry Preserves'
 'Real Mayonnaise' 'Organic Creamy Peanut Butter']

the most popular product in produce:
['Banana' 'Bag of Organic Bananas' 'Organic Strawberries' 'Strawberries'
 'Organic Hass Avocado']

the most popular product in meat seafood:
['Boneless Sk

The above dataframe prints users in the fifth group. Customers in this group would like to buy products from dairy eggs, bakery, frozen, dry goods pasta, pantry, produce, meat seafood, breakfast, deli and canned goods departments.
The favorite department is dairy eggs department.

So, as we can see, top 10 departments for these five groups are similar. But the order of these 10 departments in each group are different. LDA model could help us find the favorite department, the second favorite department…… of each group of customers. 

### Based on Aisle

#### Build Model

In [4]:
aisle_list = prior_data_da.groupby('user_id').agg({'aisle': lambda x: list(x)})

In [5]:
corps_aisle = aisle_list.aisle.tolist()

In [6]:
len(corps_aisle)

206209

In [7]:
dictionary_aisle = corpora.dictionary.Dictionary(corps_aisle)

In [8]:
corpus_aisle = [dictionary_aisle.doc2bow(corps) for corps in corps_aisle]

In [9]:
# num_topics = 5
# chunksize = 70000
# passes = 20
# iterations = 100
# eval_every = None  # Don't evaluate model perplexity, takes too much time.

# # Make a index to word dictionary.
# temp_aisle = dictionary_aisle[0]  # This is only to "load" the dictionary.
# id2word_aisle = dictionary_aisle.id2token
# #alpha='auto', eta='auto', \
# %time model_aisle = LdaMulticore(corpus=corpus_aisle, id2word=id2word_aisle, chunksize=chunksize, \
#                        iterations=iterations, num_topics=num_topics, \
#                        passes=passes, eval_every=eval_every, eta='auto', workers = 4)

In [13]:
# save the model to disk
# filename2 = 'LDA_model_aisle.sav'
# pickle.dump(model_aisle, open(filename2, 'wb'))

In [11]:
#load the model from disk
filename2 = 'LDA_model_aisle.sav'
model2 = pickle.load(open(filename2, 'rb'))

In [12]:
len(prior_data_da.aisle_id.unique())

134

#### Print Top 10 Departments for Each Group

In [14]:
model2.show_topics()

[(0,
  u'0.052*"frozen meals" + 0.042*"soft drinks" + 0.042*"ice cream ice" + 0.035*"chips pretzels" + 0.034*"paper goods" + 0.032*"packaged cheese" + 0.022*"cereal" + 0.021*"bread" + 0.019*"crackers" + 0.019*"hot dogs bacon sausage"'),
 (1,
  u'0.135*"water seltzer sparkling water" + 0.088*"fresh fruits" + 0.067*"refrigerated" + 0.049*"chips pretzels" + 0.044*"energy granola bars" + 0.042*"yogurt" + 0.039*"candy chocolate" + 0.035*"tea" + 0.035*"soy lactosefree" + 0.025*"packaged vegetables fruits"'),
 (2,
  u'0.209*"fresh fruits" + 0.136*"yogurt" + 0.086*"milk" + 0.064*"packaged vegetables fruits" + 0.049*"fresh vegetables" + 0.045*"packaged produce" + 0.037*"packaged cheese" + 0.033*"bread" + 0.021*"eggs" + 0.020*"cereal"'),
 (3,
  u'0.100*"fresh fruits" + 0.085*"baby food formula" + 0.054*"packaged vegetables fruits" + 0.054*"yogurt" + 0.046*"packaged cheese" + 0.043*"energy granola bars" + 0.038*"fresh vegetables" + 0.031*"chips pretzels" + 0.030*"crackers" + 0.027*"frozen breakfa

In [15]:
topics_matrix2 = model2.show_topics(formatted=False, num_words=10)
topics_matrix2 = np.array(topics_matrix2, dtype=object)

In [16]:
for i in range(len(topics_matrix2)):
    print [str(topics_matrix2[i][1][index][0]) for index in range(10)]
    print 

['frozen meals', 'soft drinks', 'ice cream ice', 'chips pretzels', 'paper goods', 'packaged cheese', 'cereal', 'bread', 'crackers', 'hot dogs bacon sausage']

['water seltzer sparkling water', 'fresh fruits', 'refrigerated', 'chips pretzels', 'energy granola bars', 'yogurt', 'candy chocolate', 'tea', 'soy lactosefree', 'packaged vegetables fruits']

['fresh fruits', 'yogurt', 'milk', 'packaged vegetables fruits', 'fresh vegetables', 'packaged produce', 'packaged cheese', 'bread', 'eggs', 'cereal']

['fresh fruits', 'baby food formula', 'packaged vegetables fruits', 'yogurt', 'packaged cheese', 'energy granola bars', 'fresh vegetables', 'chips pretzels', 'crackers', 'frozen breakfast']

['fresh vegetables', 'fresh fruits', 'packaged vegetables fruits', 'fresh herbs', 'soy lactosefree', 'packaged cheese', 'frozen produce', 'canned jarred vegetables', 'eggs', 'yogurt']



#### Printed Popular Products for Each Group based on Aisle

In [17]:
top_words2 = [['frozen meals', 'soft drinks', 'ice cream ice', 'chips pretzels', 'paper goods', 'packaged cheese', 'cereal', 'bread', 'crackers', 'hot dogs bacon sausage'],

['water seltzer sparkling water', 'fresh fruits', 'refrigerated', 'chips pretzels', 'energy granola bars', 'yogurt', 'candy chocolate', 'tea', 'soy lactosefree', 'packaged vegetables fruits'],

['fresh fruits', 'yogurt', 'milk', 'packaged vegetables fruits', 'fresh vegetables', 'packaged produce', 'packaged cheese', 'bread', 'eggs', 'cereal'],

['fresh fruits', 'baby food formula', 'packaged vegetables fruits', 'yogurt', 'packaged cheese', 'energy granola bars', 'fresh vegetables', 'chips pretzels', 'crackers', 'frozen breakfast'],

['fresh vegetables', 'fresh fruits', 'packaged vegetables fruits', 'fresh herbs', 'soy lactosefree', 'packaged cheese', 'frozen produce', 'canned jarred vegetables', 'eggs', 'yogurt']]


In [18]:
len(top_words2)

5

In [19]:
docs_topic2 = []

for i in range(len(corpus_aisle)):
    a = model2.get_document_topics(corpus_aisle[i])
    a.sort(key = lambda x: x[1],reverse=True)
    docs_topic2.append(a[0][0])

In [20]:
aisle_list['topic'] = docs_topic2
aisle_list = aisle_list.reset_index()

In [21]:
aisle_list

,user_id,aisle,topic
0,1,"[soft drinks, soy lactosefree, popcorn jerky, ...",1
1,2,"[popcorn jerky, fresh fruits, lunch meat, cond...",2
2,3,"[fresh fruits, soy lactosefree, fresh fruits, ...",3
3,4,"[bread, water seltzer sparkling water, frozen ...",0
4,5,"[honeys syrups nectars, asian foods, fresh veg...",4
5,6,"[fresh vegetables, poultry counter, packaged v...",4
6,7,"[yogurt, packaged vegetables fruits, bread, fr...",1
7,8,"[packaged vegetables fruits, fresh vegetables,...",4
8,9,"[yogurt, spices seasonings, packaged vegetable...",3
9,10,"[fresh vegetables, fresh fruits, fresh vegetab...",4


In [38]:
aisle_list.to_csv('aisle_list.csv')

#### Popular Products for Group0

In [22]:
grouped2 = aisle_list.groupby('topic')
aisle_group = {}
#user_group = {}
for i in range(len(topics_matrix2)):
    aisle_group[i] = grouped2.get_group(i)
    #user_group[i] = department_group[i].user_id

In [23]:
aisle_group[0]

,user_id,aisle,topic
3,4,"[bread, water seltzer sparkling water, frozen ...",0
13,14,"[fresh vegetables, bread, spices seasonings, c...",0
16,17,"[water seltzer sparkling water, soft drinks, w...",0
18,19,"[chips pretzels, frozen appetizers sides, pape...",0
22,23,"[baking ingredients, food storage, canned jarr...",0
23,24,"[juice nectars, baking ingredients, juice nect...",0
24,25,"[instant foods, frozen meals, bread, buns roll...",0
28,29,"[juice nectars, cookies cakes, ice cream ice, ...",0
30,31,"[soy lactosefree, soy lactosefree, cream, tea,...",0
32,33,"[water seltzer sparkling water, crackers, milk...",0


In [24]:
print_top_product(top_words2, 0, aisle_group, 'aisle')

the most popular product in frozen meals:
['Macaroni & Cheese' 'Chicken Pot Pie' 'Pesto Tortellini  Bowls'
 'Broccoli & Cheddar Bake Meal Bowl' 'Kitchen Cheese Enchilada']

the most popular product in soft drinks:
['Soda' 'Fridge Pack Cola' 'Cola' 'Coke Classic' 'Ginger Ale']

the most popular product in ice cream ice:
['Chocolate Ice Cream' 'Vanilla Ice Cream'
 'Chocolate Chip Cookie Dough Ice Cream' 'Natural Vanilla Ice Cream'
 'Half Baked\xc2\xae Ice Cream']

the most popular product in chips pretzels:
['Scoops! Tortilla Chips' 'Original Restaurant Style Tortilla Chips'
 'Classic Potato Chips' 'Hint of Lime Flavored Tortilla Chips'
 'Original Potato Chips']

the most popular product in paper goods:
['Ultra Soft Facial Tissues' 'Select-A-Size White Paper Towels'
 'White Giant Paper Towel Rolls'
 'Select-A-Size Paper Towels, White, 2 Huge Rolls = 5 Regular Rolls  Towels/Napkins'
 'Sensitive Toilet Paper']

the most popular product in packaged cheese:
['Sharp Cheddar Cheese' 'Natural S

This is the top 10 words in this group: ['frozen meals', 'soft drinks', 'ice cream ice', 'chips pretzels', 'paper goods', 'packaged cheese', 'cereal', 'bread', 'crackers', 'hot dogs bacon sausage']
So, we can see customers in this group would like to buy frozen means. 

#### Popular Products for Group1

In [25]:
aisle_group[1]

,user_id,aisle,topic
0,1,"[soft drinks, soy lactosefree, popcorn jerky, ...",1
6,7,"[yogurt, packaged vegetables fruits, bread, fr...",1
14,15,"[nuts seeds dried fruit, popcorn jerky, nuts s...",1
19,20,"[crackers, popcorn jerky, fresh fruits, fresh ...",1
20,21,"[eggs, eggs, breakfast bakery, yogurt, chips p...",1
25,26,"[refrigerated pudding desserts, baking ingredi...",1
26,27,"[coffee, refrigerated, refrigerated, refrigera...",1
29,30,"[water seltzer sparkling water, candy chocolat...",1
37,38,"[water seltzer sparkling water, soy lactosefre...",1
51,52,"[packaged produce, nuts seeds dried fruit, nut...",1


In [145]:
print_top_product(top_words2, 1, aisle_group, 'aisle')

the most popular product in water seltzer sparkling water:
['Sparkling Water Grapefruit' 'Spring Water' 'Lime Sparkling Water'
 'Sparkling Lemon Water' 'Sparkling Natural Mineral Water']

the most popular product in fresh fruits:
['Banana' 'Bag of Organic Bananas' 'Organic Strawberries'
 'Organic Hass Avocado' 'Organic Fuji Apple']

the most popular product in refrigerated:
['100% Raw Coconut Water' 'Organic Raw Kombucha Gingerade'
 'Trilogy Kombucha Drink' 'Original Orange Juice'
 'Synergy Organic Kombucha Gingerberry']

the most popular product in chips pretzels:
['Organic Simply Naked Pita Chips' 'Lightly Salted Baked Snap Pea Crisps'
 'Organic Tortilla Chips' 'Pub Mix' 'Sea Salt & Vinegar Potato Chips']

the most popular product in energy granola bars:
["Crunchy Oats 'n Honey Granola Bars"
 'Gluten Free Dark Chocolate Chunk Chewy with a Crunch Granola Bars'
 'Oats & Honey Gluten Free Granola'
 'Sweet & Salty Nut Granola Bars Peanut'
 'Gluten Free Peanut Butter Dark Chocolate Chewy 

Customers in thi group would like to products from aisle 'water seltzer sparkling water', 'fresh fruits', 'refrigerated', 'chips pretzels', 'energy granola bars', 'yogurt', 'candy chocolate', 'tea', 'soy lactosefree' and 'packaged vegetables fruits'. This group of customers look healthier than the first group of customers. Instead of buying soft dringks or ice creas, they would like to buy more sparkling watter, fresh fruits and yougurts.

#### Popular Products for Group2

In [61]:
aisle_group[2]

,user_id,aisle,topic
1,2,"[popcorn jerky, fresh fruits, lunch meat, cond...",2
42,43,"[cream, hot dogs bacon sausage, fresh fruits, ...",2
55,56,"[other creams cheeses, other creams cheeses, i...",2
70,71,"[breakfast bakery, specialty cheeses, cream, m...",2
71,72,"[milk, juice nectars, fresh fruits, paper good...",2
78,79,"[fresh vegetables, fresh fruits, bread, eggs, ...",2
79,80,"[frozen breakfast, fresh fruits, fresh fruits,...",2
80,81,"[fresh fruits, fresh vegetables, packaged prod...",2
86,87,"[milk, specialty wines champagnes, butter, bre...",2
90,91,"[fresh fruits, energy granola bars, candy choc...",2


In [146]:
print_top_product(top_words2, 2, aisle_group, 'aisle')

the most popular product in fresh fruits:
['Bag of Organic Bananas' 'Banana' 'Organic Strawberries' 'Strawberries'
 'Organic Hass Avocado']

the most popular product in yogurt:
['Total 2% with Strawberry Lowfat Greek Strained Yogurt'
 'Total 2% Lowfat Greek Strained Yogurt with Peach'
 'Total 2% Lowfat Greek Strained Yogurt With Blueberry'
 'Total 2% All Natural Greek Strained Yogurt with Honey'
 'Total 2% Greek Strained Yogurt with Cherry 5.3 oz']

the most popular product in milk:
['Organic Whole Milk' 'Organic Reduced Fat 2% Milk'
 'Organic Reduced Fat Milk' 'Organic Fat Free Milk' 'Fat Free Milk']

the most popular product in packaged vegetables fruits:
['Organic Baby Spinach' 'Organic Raspberries' 'Organic Blueberries'
 'Seedless Red Grapes' 'Organic Baby Carrots']

the most popular product in fresh vegetables:
['Cucumber Kirby' 'Organic Yellow Onion' 'Organic Zucchini'
 'Organic Garlic' 'Organic Cucumber']

the most popular product in packaged produce:
['Raspberries' 'Hass Avocad

This group of customers lead even healthier life than the second group of custoemrs. The favorite aisle of this group of customer is fresh frutis. The top 10 aisle are ['fresh fruits', 'yogurt', 'milk', 'packaged vegetables fruits', 'fresh vegetables', 'packaged produce', 'packaged cheese', 'bread', 'eggs', 'cereal']. Compared to the second group of customers. There is no canides. It looks customers from second and third group have some similar favorite aisles, but products they would like to buy have some differences. For example, customers in third group prefer to total 2% yogurt, while customers in the second group also like total 0% fat milk.  


#### Popular Products for Group3

In [62]:
aisle_group[3]

,user_id,aisle,topic
2,3,"[fresh fruits, soy lactosefree, fresh fruits, ...",3
8,9,"[yogurt, spices seasonings, packaged vegetable...",3
31,32,"[lunch meat, lunch meat, fresh herbs, fresh ve...",3
39,40,"[fresh fruits, bread, fresh fruits, trail mix ...",3
43,44,"[hot cereal pancake mixes, packaged vegetables...",3
46,47,"[baking ingredients, yogurt, fresh fruits, pac...",3
50,51,"[seafood counter, packaged cheese, fresh dips ...",3
54,55,"[butter, chips pretzels, fruit vegetable snack...",3
58,59,"[soy lactosefree, fresh fruits, fresh fruits, ...",3
59,60,"[bread, hot cereal pancake mixes, hot cereal p...",3


In [150]:
print_top_product(top_words2, 3, aisle_group, 'aisle')

the most popular product in fresh fruits:
['Banana' 'Bag of Organic Bananas' 'Organic Strawberries'
 'Organic Hass Avocado' 'Organic Avocado']

the most popular product in baby food formula:
['Baby Food Stage 2 Blueberry Pear & Purple Carrot'
 'Spinach Peas & Pear Stage 2 Baby Food'
 'Broccoli & Apple Stage 2 Baby Food'
 'Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food'
 'Gluten Free SpongeBob Spinach Littles']

the most popular product in packaged vegetables fruits:
['Organic Baby Spinach' 'Organic Raspberries' 'Organic Blueberries'
 'Seedless Red Grapes' 'Organic Baby Carrots']

the most popular product in yogurt:
['YoKids Squeezers Organic Low-Fat Yogurt, Strawberry'
 'Total 2% with Strawberry Lowfat Greek Strained Yogurt'
 'YoKids Blueberry & Strawberry/Vanilla Yogurt' 'YoBaby Peach Pear Yogurt'
 'Total Greek Strained Yogurt']

the most popular product in packaged cheese:
['Organic Whole String Cheese'
 'Organic Sticks Low Moisture Part Skim Mozzarella 

For the fourth group of customers, the top aisles are ['fresh fruits', 'baby food formula', 'packaged vegetables fruits', 'yogurt', 'packaged cheese', 'energy granola bars', 'fresh vegetables', 'chips pretzels', 'crackers', 'frozen breakfast']. Baby food formula is the second most popular aisle for this group of customers. So we can assume that most of customers of this group have babies and recommend some baby products for this group. 

#### Popular Products for Group4

In [63]:
aisle_group[4]

,user_id,aisle,topic
4,5,"[honeys syrups nectars, asian foods, fresh veg...",4
5,6,"[fresh vegetables, poultry counter, packaged v...",4
7,8,"[packaged vegetables fruits, fresh vegetables,...",4
9,10,"[fresh vegetables, fresh fruits, fresh vegetab...",4
10,11,"[yogurt, yogurt, yogurt, tofu meat alternative...",4
11,12,"[fresh vegetables, packaged vegetables fruits,...",4
12,13,"[baking ingredients, fresh vegetables, fresh v...",4
15,16,"[packaged vegetables fruits, fresh fruits, fre...",4
17,18,"[digestion, soy lactosefree, water seltzer spa...",4
21,22,"[fresh vegetables, fresh vegetables, fresh veg...",4


In [151]:
print_top_product(top_words2, 4, aisle_group, 'aisle')

the most popular product in fresh vegetables:
['Organic Garlic' 'Organic Yellow Onion' 'Organic Zucchini'
 'Cucumber Kirby' 'Carrots']

the most popular product in fresh fruits:
['Banana' 'Bag of Organic Bananas' 'Organic Hass Avocado'
 'Organic Strawberries' 'Organic Avocado']

the most popular product in packaged vegetables fruits:
['Organic Baby Spinach' 'Organic Raspberries' 'Organic Grape Tomatoes'
 'Organic Baby Arugula' 'Organic Blueberries']

the most popular product in fresh herbs:
['Organic Cilantro' 'Organic Italian Parsley Bunch' 'Bunched Cilantro'
 'Organic Basil' 'Organic Thyme']

the most popular product in soy lactosefree:
['Organic Unsweetened Almond Milk' 'Unsweetened Almondmilk'
 'Organic Coconut Milk' 'Unsweetened Original Almond Breeze Almond Milk'
 'Vanilla Almond Breeze Almond Milk']

the most popular product in packaged cheese:
['Grated Parmesan' 'Feta Cheese Crumbles' 'Organic Whole String Cheese'
 'Shredded Parmesan' 'Sharp Cheddar Cheese']

the most popular p

The top 10 aisles of this group customers are ['fresh vegetables', 'fresh fruits', 'packaged vegetables fruits', 'fresh herbs', 'soy lactosefree', 'packaged cheese', 'frozen produce', 'canned jarred vegetables', 'eggs', 'yogurt']]. We can see that there is no meat or candies for this group. So we can assume that this group of customers could be some old people or some people not allowed to get sugar. So we could recommend some products without sugar and more generally, healthier food or organic food might be popular in this group. 